# 🏥 AI-Doctor Workshop - Kun jij een computer leren om ziektes te herkennen?

Welkom bij deze workshop over kunstmatige intelligentie! We gaan een computer leren om röntgenfoto's van longen te bekijken en te bepalen of iemand longontsteking heeft of niet. 

## 🚀 Hoe werkt dit?

Dit bestand is een *Jupyter Notebook*: een soort digitaal werkboek waar tekst (zoals dit) en programmeercode door elkaar staan. Je kunt de grijze vakjes met code laten draaien door erop te klikken en dan **Shift + Enter** te drukken.

**⚠️ BELANGRIJK:** Begin met het draaien van de cel hieronder, anders werkt niets!

In [ ]:
# 📦 Alle tools en bibliotheken laden die we nodig hebben
# Dit is zoals het uitpakken van een gereedschapskist

import pip
import numpy as np
import os
from glob import glob
import torch
from PIL import Image
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import torch.nn.functional as F

print("🔧 Installeren van speciale medische data tools...")
%pip install medmnist
from medmnist import PneumoniaMNIST

print("🔧 Installeren van tools voor medische beelden...")
%pip install SimpleITK
import SimpleITK as sitk

print("📂 Downloaden van voorbeeld bestanden...")    
!git clone https://github.com/clarastegehuis/machine_learning_medical_data_workshop/

print("✅ Alles klaar! Laten we beginnen!")

# 🤖 Hoe kan een computer beelden "zien"?

Stel je voor: je bent dokter en elke dag moet je tientallen röntgenfoto's bekijken om te zien of patiënten ziek zijn. Na een paar uur word je moe en kun je dingen over het hoofd zien. Maar wat als een computer je kon helpen die nooit moe wordt?

## 🎯 Onze missie vandaag:
We gaan een slimme computer maken die automatisch kan zien of iemand longontsteking heeft op een röntgenfoto.

**Fun fact:** Computers zijn soms zelfs beter dan mensen in het herkennen van bepaalde ziektes op medische foto's! 🤯

## 🖼️ Hoe "ziet" een computer eigenlijk een foto?

Voor jou is een foto gewoon een foto. Maar voor een computer is het heel anders!

**Voor mensen:** 👁️ Wij zien vormen, kleuren, objecten  
**Voor computers:** 🔢 Een computer ziet alleen maar getallen

Elke foto wordt opgeslagen als een gigantische tabel vol getallen. Elk vakje in die tabel heet een **pixel** en bevat een getal dat zegt hoe licht of donker dat stukje van de foto is.

- **Zwart-wit foto:** 1 tabel met getallen van 0 (zwart) tot 255 (wit)
- **Kleurenfoto:** 3 tabellen: één voor rood, één voor groen, en één voor blauw

Laten we dit eens bekijken met een echte röntgenfoto:

In [ ]:
# 🛠️ Handige functies om afbeeldingen te openen en te bekijken

def open_img(path):
    """Deze functie opent een afbeelding - zoals het openen van een boek!"""
    if path.endswith('.png'):
        return np.array(Image.open(path).convert('L'))
    elif path.endswith('.mhd'):
        return sitk.GetArrayFromImage(sitk.ReadImage(path))[32,:,:] # return 1 slice of the image

def visualize(img, clim=[-300,450]):
    """Deze functie laat een afbeelding zien - zoals het aandoen van het licht!"""
    plt.imshow(img, cmap='gray', clim=clim)
    plt.axis('off')  # Geen getallen op de randen
    plt.show()
    
print("🎨 Functies geladen! Klaar om afbeeldingen te bekijken!")

In [ ]:
# 📸 Laten we onze eerste medische afbeelding bekijken!

print("🏥 Laden van een röntgenfoto van de ribbenkast...")

# Het pad naar onze afbeelding
img_path = '/content/machine_learning_medical_data_workshop/TEV1P1CTI.mhd'
img = open_img(img_path)

print("✨ Hier is de röntgenfoto:")
visualize(img)

print("\n🤔 Dit is hoe een computer deze foto 'ziet', een tabel vol getallen")

## 🔍 Vraag: Hoeveel pixels zitten er in deze foto?

Net zoals een puzzel uit puzzelstukjes bestaat, bestaat een digitale foto uit pixels: kleine, vierkante blokjes. Laten we eens kijken hoeveel "puzzelstukjes" onze röntgenfoto heeft:

In [ ]:
# 🧮 Laten we de pixels tellen!

print("📐 De afmetingen van onze foto zijn:")
vorm = np.shape(img)
print(f"   Hoogte: {vorm[0]} pixels")
print(f"   Breedte: {vorm[1]} pixels")

# Bereken het totale aantal pixels
totaal_pixels = vorm[0] * vorm[1]
print(f"\n🎯 Totaal aantal pixels: {totaal_pixels:,}")
print(f"   Dat zijn meer dan {totaal_pixels/1000:.0f} duizend kleine puntjes!")

print(f"\n🤯 Stel je voor: de computer moet al deze {totaal_pixels:,} getallen bekijken om te begrijpen wat er in de foto staat!")

## 🔍 De kracht van "Convoluties": Hoe computers patronen herkennen!

Stel je voor dat je een detective bent die naar aanwijzingen zoekt in een foto. Je zou een vergrootglas gebruiken om elk deel van de foto zorgvuldig te bekijken, toch?

Een **convolutie** werkt precies zo! Het is als een slim vergrootglas dat over de hele foto schuift en op elk plekje kijkt: "Hmm, past dit patroon hier?"

### 🕵️ Hoe werkt het?

1. **De Kernel:** Dit is ons "vergrootglas": een kleine tabel met getallen die een bepaald patroon beschrijft
2. **Het Schuiven:** We bewegen dit vergrootglas over elke pixel van de foto  
3. **Het Vergelijken:** Op elke plek kijken we hoe goed het patroon overeenkomt
4. **Het Resultaat:** We krijgen een nieuwe foto die laat zien waar we ons patroon gevonden hebben!

![Convolutie Animatie](https://upload.wikimedia.org/wikipedia/commons/0/04/Convolution_arithmetic_-_Padding_strides.gif?20190413174630)

### 🎯 Wat kunnen we ermee vinden?
- **Verticale lijnen** (zoals de randen van botten)
- **Horizontale lijnen** (zoals ribben)
- **Vervaging wegwerken** (foto's scherper maken)
- **Objecten verschuiven** (cool trucje!)
- **Veel meer...**

Laten we het uitproberen! 🚀

In [ ]:
# 🎬 Convolutie-animatie

def apply_conv(image, kernel, iter=1):
    """
    Deze functie past een convolutie toe op een afbeelding.
    Het is als een filmpje dat laat zien hoe de foto verandert!
    """
    image, kernel = torch.from_numpy(image).float(), torch.from_numpy(kernel).float()
    img_shape, kernel_shape = image.shape, kernel.shape
    fig, ax = plt.subplots(1,1, figsize=(8, 6))
    
    print(f"🎭 Bezig met {iter} convolutie(s) toepassen...")
    
    for level in range(iter):
        image = F.conv2d(image.reshape(1,1, img_shape[0], img_shape[1]),
                         kernel.reshape(1,1, kernel_shape[0], kernel_shape[1]),
                         padding='same').squeeze()
        ax.clear()
        ax.imshow(image.numpy(), cmap='gray', clim=[-300,450])
        ax.set_title(f'🔄 Convolutie #{level+1} - Zie hoe de foto verandert!', fontsize=14)
        ax.axis('off')
        display(fig)
        clear_output(wait=True)
        plt.pause(0.3)  # Wat langzamer zodat je het beter kunt zien
    
    plt.close()
    print(f"✨ Klaar! De convolutie is {iter} keer toegepast!")

## 🎮 Experiment 1: De verschuiving

Nu wordt het leuk! We gaan onze eerste "kernel" (het vergrootglas) uitproberen. Deze kernel kan iets heel bijzonders: **hij verschuift de hele foto naar links!**

### 🤔 Denk na:
- Wat zou er gebeuren als we dit 20 keer achter elkaar doen?
- Zou de foto helemaal verdwijnen aan de linkerkant?

**Druk op de cel hieronder en kijk wat er gebeurt!** 👀

In [ ]:
# 🎯 Experiment 1: Foto naar links verschuiven!

print("🔧 Maken van een 'links-verschuif' kernel...")
print("   Deze 3x3 tabel zegt: neem elke pixel en zet hem één plekje naar links!")

# Onze kernel:
kernel = np.array([[0, 0, 0],    # Bovenste rij: doe niets
                   [0, 0, 1],    # Middelste rij: verschuif naar links (1 betekent "kopieer hier")
                   [0, 0, 0]])   # Onderste rij: doe niets

print("\n🎬 En... actie! Kijk hoe de foto steeds verder naar links schuift:")
print("   (Dit kan even duren - blijf kijken!)")

# Pas het 20 keer toe - zie het schuiven!
n_iters = 20
apply_conv(img, kernel, n_iters)

print("\n🎉 Mooi toch? De hele foto is naar links 'gelopen'!")

## 🎮 Experiment 2: Naar boven zweven

Nu proberen we iets anders - we laten de foto naar **boven** zweven! 

### 🏆 UITDAGING VOOR JOU:
Na dit experiment: **kun jij een kernel maken die de foto schuin naar rechts-onder laat bewegen?**  
(Tip: je moet de juiste plek vinden om het getal "1" neer te zetten!)

In [ ]:
# 🚁 Experiment 2: Foto Naar Boven Laten Zweven!

print("🔧 Maken van een 'omhoog-zweef' kernel...")
print("   Deze kernel zegt: til elke pixel één plekje omhoog!")

# Onze zweef-kernel:
kernel = np.array([[0, 0, 0],    # Bovenste rij: doe niets
                   [0, 0, 0],    # Middelste rij: doe niets  
                   [0, 1, 0]])   # Onderste rij: kopieer naar boven (midden)

print("\n🎬 Kijk hoe de foto omhoog zweeft als een ballon! 🎈")

# Laat het 20 keer omhoog zweven
n_iters = 20
apply_conv(img, kernel, n_iters)

print("\n🤔 Begrijp je het patroon? De positie van het getal '1' bepaalt waar de foto naartoe beweegt!")

## 🐎 SUPER UITDAGING: De paardensprong

Heb je wel eens schaak gespeeld? Een paard beweegt in een L-vorm: 2 vakjes in één richting, dan 1 vakje opzij.

### 🎯 Jouw missie:
Maak een 5×5 kernel die de foto een **paardensprong** laat maken:
- 2 pixels omhoog
- 1 pixel naar links

**Hint:** 🤫 
- Alle vakjes in de kernel zijn 0, behalve ÉÉN vakje
- Dat ene vakje moet een 1 worden
- Denk goed na waar je die 1 neerzet!

**Extra uitdaging:** Kun je ook andere schaakstukken namaken? Een toren (recht), een loper (schuin)?

In [ ]:
# 🐎 Jouw Paardensprong Kernel - Kun jij het oplossen?

print("🏁 Dit is jouw uitdaging! Verander de kernel hieronder:")
print("   Zet een '1' op de juiste plek om een paardensprong te maken!")
print("   (2 omhoog, 1 naar links)")

# VERANDER DEZE KERNEL - zet ergens een 1 neer!
kernel = np.array([[0, 0, 0, 0, 0],    # Rij 1
                   [0, 0, 0, 0, 0],    # Rij 2  
                   [0, 0, 0, 0, 0],    # Rij 3 (midden)
                   [0, 0, 0, 0, 0],    # Rij 4
                   [0, 0, 0, 0, 0]])   # Rij 5

print("\n🎬 Laten we kijken wat jouw kernel doet:")

n_iters = 20
apply_conv(img, kernel, n_iters)

print("\n🤔 Lukte het? Als de foto niet bewoog, probeer dan een andere plek!")
print("💡 Tip: denk aan de kernel als een kaart: waar wil je dat de pixel naartoe gaat?")

## 🌫️ Experiment 3: De waas(Smoothing)

Deze kernel doet iets heel anders dan verschuiven - hij maakt de foto **waziger**! Het is alsof je door een beslagen raam kijkt.

### 🤓 Hoe werkt het?
In plaats van een pixel te kopiëren, neemt deze kernel het **gemiddelde** van een pixel en al zijn buren. Hierdoor verdwijnen scherpe randen en wordt alles zachter.

### 🔬 Experimenteer!
Probeer de getallen in de kernel te veranderen - wat gebeurt er dan?

In [ ]:
# 🌫️ De waas!

print("🔧 Maken van een 'smoothing' kernel...")
print("   Deze kernel neemt van elke pixel het gemiddelde met zijn buren!")

# Een smoothing kernel - let op de speciale getallen!
kernel = np.array([[1, 2, 1],     # Hoeken tellen voor 1, zijkanten voor 2
                   [2, 4, 2],     # Het midden telt het zwaarst (4)  
                   [1, 2, 1]]) * 1/16  # Deel door 16 om het gemiddelde te krijgen

print("🧮 Wiskundig trucje: alle getallen bij elkaar = 16")
print("   Daarom delen we door 16 - zo krijgen we het gemiddelde!")

print("\n🎬 Kijk hoe de foto steeds waziger wordt:")

n_iters = 20

apply_conv(img, kernel, n_iters)

print("\n💡 Cool detail: medische scanners gebruiken dit ook om ruis weg te halen!")
print("🔬 Probeer de getallen in de kernel te veranderen - wat gebeurt er dan?")

## 🔍 Experiment 4: De lijndetective!

Deze kernel kan **verticale lijnen** vinden in de foto - zoals de randen van botten!

### 🕵️ Hoe werkt de detective?
- **+1 waarden:** "Zoek naar lichte plekjes hier"
- **-1 waarden:** "Zoek naar donkere plekjes hier"  
- **0 waarden:** "Dit interesseert me niet"

Verticale lijnen hebben lichte en donkere kanten naast elkaar - precies wat deze kernel zoekt!

### 🏆 NIEUWE UITDAGING:
**Kun jij een kernel maken die horizontale lijnen vindt?** En wat gebeurt er als je alle +1 en -1 omwisselt?

In [ ]:
# 🔍 De Verticale Lijndetective

print("🕳️ Maken van een 'verticale lijn detector'...")
print("   Deze kernel zoekt naar verticale randen - zoals de zijkanten van botten!")

# Verticale lijn detector
kernel = np.array([[1,  0, -1],   # Links licht (+1), rechts donker (-1)
                   [1,  0, -1],   # Dit patroon herhaalt zich
                   [1,  0, -1]])  # Zo vind je verticale lijnen!

print("\n🧠 Slim he? Deze kernel zegt:")
print("   'Als links licht is en rechts donker, dan is hier een verticale lijn!'")

print("\n🎬 Kijk hoe de verticale lijnen oplichten:")

# Deze hoeven we maar 1 keer toe te passen!
n_iters = 1
apply_conv(img, kernel, n_iters)

print("\n✨ Zie je de verticale randen van de ribben oplichten?")
print("\n🏆 UITDAGING: Probeer nu een horizontale lijndetector te maken")

## 🧠 Van simpele trucjes naar echte AI!

**Plot twist!** 🎬 Wat we net gedaan hebben zijn de **bouwstenen** van kunstmatige intelligentie!

### 🏗️ Hoe bouw je een AI-dokter?

1. **Stap 1:** Begin met simpele convoluties (zoals we net deden)
2. **Stap 2:** Stapel er tientallen, soms honderden van bovenop elkaar!
3. **Stap 3:** Laat de computer zelf uitvogelen welke kernels het beste werken
4. **Stap 4:** Train het met duizenden voorbeelden van zieke en gezonde longen

### 🤯 Het belangrijkste deel:
De computer bedenkt zelf de beste kernels! Wij geven hem alleen voorbeelden en hij leert: 
*"Ah, als ik deze kernel gebruik, kan ik longontsteking herkennen!"*

**Dit noemen we een Convolutional Neural Network (CNN)** - een neuraal netwerk dat gespecialiseerd is in het begrijpen van afbeeldingen.

Klaar voor de echte AI? 🚀

## 🏥 Onze missie: De AI-dokter trainen

**Het verhaal:** Er zijn duizenden röntgenfoto's van longen: sommige van gezonde mensen, andere van mensen met longontsteking. Voor een dokter is het soms moeilijk om het verschil te zien, vooral na een lange dag vol patiënten.

### 🎯 Ons Doel:
We gaan een slimme computer trainen die in één oogopslag kan zeggen:
- 💚 **"Deze longen zijn gezond!"**  
- 🔴 **"Deze patient heeft longontsteking!"**

### 📊 Onze Data:
We gebruiken de **PneumoniaMNIST dataset** - dat zijn duizenden kleine röntgenfoto's die al gelabeld zijn door echte dokters.

Laten we beginnen! 🚀

In [ ]:
# 📦 Downloaden van de Longfoto Dataset!

print("🏥 Bezig met downloaden van duizenden röntgenfoto's...")
print("   (Dit kan even duren - we krijgen echte medische data binnen!)")

import medmnist
%pip install monai

# Download de pneumonia dataset - foto's van longen!
dataset = medmnist.PneumoniaMNIST(split="train", download=True)

print(f"✅ Gelukt! We hebben {len(dataset)} röntgenfoto's gedownload!")
print("🔬 Elke foto is al beoordeeld door echte dokters:")
print("   - Label 0 = Gezonde longen 💚")  
print("   - Label 1 = Longontsteking 🔴")

In [ ]:
# 🛠️ Het Voorbereiden van de Data voor onze AI

import monai

# Een speciale klasse die de data in het juiste formaat zet voor onze AI
class MedMNISTData(monai.data.Dataset):
    """
    Deze klasse zorgt ervoor dat onze AI de foto's goed kan begrijpen.
    Het is zoals het vertalen van een boek naar een taal die de computer snapt!
    """
    
    def __init__(self, datafile, transform=None):
        self.data = datafile
        self.transform = transform
        
    def __getitem__(self, index):
        # Maak een dictionary met 'img' en 'label' 
        image = torch.from_numpy(np.array(self.data[index][0])).float()
        if self.transform:
            image = self.transform(image)
        return {'img': image, 'label': self.data[index][1]}
    
    def __len__(self):
        return len(self.data)

print("🔧 Data-klasse gemaakt! Nu kan onze AI de foto's begrijpen!")

In [ ]:
# 🎨 Functie om Longfoto's Mooi te Laten Zien

def visualize_sample(sample):
    """
    Deze functie laat een longfoto zien met de juiste diagnose erbij
    """
    plt.figure(figsize=(6, 6))
    plt.imshow(sample['img'], 'gray')
    
    # Geef elke foto een duidelijke titel
    if sample['label'] == 1:
        plt.title('🔴 Patient met longontsteking', fontsize=14, color='red')
    else:
        plt.title('💚 Gezonde patient', fontsize=14, color='green')
    
    # Geen vervelende nummertjes op de zijkanten
    plt.xticks([]) 
    plt.yticks([]) 
    plt.show()

print("🎨 Visualisatie functie klaar. Nu kunnen we mooie foto's bekijken!")

In [ ]:
# 🔧 Data klaarmaken voor Ttraining

print("🔄 Bezig met data normaliseren...")
print("   (Dit zorgt ervoor dat alle getallen tussen -1 en +1 liggen)")

from monai.transforms import NormalizeIntensity

# Deze transformatie maakt de pixel-waarden geschikt voor AI training
data_transform = NormalizeIntensity(subtrahend=.5, divisor=.5)

# Maak onze training dataset
train_dataset = MedMNISTData(dataset, transform=data_transform)

print(f"✅ Training dataset klaar! {len(train_dataset)} foto's zijn gereed voor AI training!")
print("🎓 Nu kan onze computer leren wat het verschil is tussen zieke en gezonde longen!")

## 🔍 Onderzoekstijd: Verken de dataset!

**Tijd om detective te spelen!** 🕵️‍♀️ We hebben nu 4,708 röntgenfoto's die al door echte dokters zijn beoordeeld.

### 📋 Je onderzoeksopdrachten:

1. **Bekijk verschillende foto's:** Probeer `visualize_sample(train_dataset[k])` met verschillende getallen voor `k` (van 0 tot 4707)
2. **Zoek patronen:** Kun je het verschil zien tussen gezonde en zieke longen?
3. **Word nieuwsgierig:** Welke foto's zijn moeilijk om te beoordelen?

### 🧪 Experiment hieronder:
Verander het getal tussen de [] om verschillende foto's te bekijken

In [ ]:
# 🔬 JOUW ONDERZOEK: Verken de dataset!

print("🕵️ Tijd om de dataset te onderzoeken!")
print("   Verander het getal hieronder om verschillende foto's te bekijken:")

# VERANDER DIT GETAL om verschillende foto's te zien! (0 tot 4707)
foto_nummer = 42

print(f"\n📸 Bekijken van foto nummer {foto_nummer}:")
visualize_sample(train_dataset[foto_nummer])

print(f"\n🏷️ Deze foto heeft label: {train_dataset[foto_nummer]['label'][0]}")
if train_dataset[foto_nummer]['label'][0] == 0:
    print("   💚 Dat betekent: GEZONDE longen!")
else:
    print("   🔴 Dat betekent: LONGONTSTEKING!")

print(f"\n🎲 Probeer verschillende getallen tussen 0 en {len(train_dataset)-1}!")
print("💡 Tip: Kun je het verschil zien tussen gezonde en zieke longen?")

## 📊 Belangrijke vraag: Is onze dataset gebalanceerd?

Stel je voor: je wilt leren om appels en peren te herkennen, maar je trainingsset heeft 1000 appels en maar 10 peren. Dan wordt je AI heel goed in appels herkennen, maar slecht in peren!

### 🤔 De Balance-Check:
We gaan kijken of onze dataset **gebalanceerd** is. Hebben we evenveel foto's van gezonde longen als van zieke longen? Of is er een grote scheefheid?

### 🎯 Waarom is dit belangrijk?
- Als er veel meer foto's van zieke longen zijn, denkt de AI misschien dat iedereen ziek is
- Als er veel meer gezonde foto's zijn, mist de AI misschien echte ziektes

Laten we het uitzoeken! 👇

In [ ]:
# 📊 Dataset Balance Check + Random Foto!

print("🎲 Eerst een willekeurige foto uit de dataset:")

# Laat een willekeurige foto zien
index = np.random.choice(np.arange(len(train_dataset)))
visualize_sample(train_dataset[index])

print(f"📸 Dit was foto nummer {index}")

print("\n🔍 Nu gaan we de hele dataset analyseren...")
print("   Tellen hoeveel foto's er zijn van elke categorie...")

# Tel alle labels
counts = {0: 0, 1: 0}
for sample in train_dataset:
    counts[sample['label'][0]] += 1

print(f"\n📊 RESULTATEN:")
print(f"   💚 Gezonde longen (label 0): {counts[0]:,} foto's")
print(f"   🔴 Longontsteking (label 1): {counts[1]:,} foto's")

# Bereken percentages
totaal = counts[0] + counts[1]
percentage_gezond = (counts[0] / totaal) * 100
percentage_ziek = (counts[1] / totaal) * 100

print(f"\n📈 PERCENTAGES:")
print(f"   💚 {percentage_gezond:.1f}% gezonde foto's")
print(f"   🔴 {percentage_ziek:.1f}% longontsteking foto's")

if abs(percentage_gezond - percentage_ziek) < 10:
    print("\n✅ Dataset is goed gebalanceerd! 👍")
else:
    print(f"\n⚠️  Dataset is scheef! Er zijn veel meer {'gezonde' if counts[0] > counts[1] else 'zieke'} foto's.")
    print("   Dit kan onze AI beïnvloeden...")

## 🧠 Tijd om onze AI-dokter te bouwen

Nu komt het spannende gedeelte - we gaan ons **Convolutional Neural Network** (CNN) bouwen! Dit is ons AI-brein dat gaat leren om longontsteking te herkennen.

### 🏗️ Onze AI-Architectuur:
1. **Laag 1:** 32 verschillende convolutie-filters (32 verschillende "vergrootglazen")
2. **Laag 2:** 64 nog slimmere filters 
3. **Hersenen:** Een neuraal netwerk dat de finale beslissing maakt

### 🤔 Breinbreker Vraag:
Onze kernels zijn 3×3 pixels groot. Als we twee van deze kernels na elkaar toepassen, **hoeveel pixels** kan onze AI dan "zien" rondom elke punt? 

**Hint:** Dit noemen we het **"receptive field"** - het gebied dat de AI kan bekijken om een beslissing te maken!

Probeer het uit te rekenen voordat je verder gaat 🧮

In [ ]:
# 🔄 Data Splitsen: Training vs Validatie

print("📚 Bezig met het maken van een test-dataset...")
print("   (Dit zijn foto's die onze AI NOG NOOIT heeft gezien!)")

# Maak een validatie dataset - dit is onze 'examens' voor de AI
val_dataset = MedMNISTData(medmnist.PneumoniaMNIST(split='val', download=False))

print(f"✅ Validatie dataset klaar: {len(val_dataset)} foto's")

print("\n🎒 Maken van data-loaders...")
print("   (Dit is hoe we de foto's netjes aan onze AI voeren)")

# DataLoaders - deze voeden onze AI met data in hapklare brokken
train_dataloader = monai.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = monai.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

print("✅ Data-loaders klaar!")
print(f"   📖 Training: {len(train_dataloader)} batches van 32 foto's")
print(f"   📝 Validatie: {len(val_dataloader)} batches van 32 foto's")

print("\n🎯 Nu kan onze AI leren van de training data en zichzelf testen op de validatie data!")

In [ ]:
# 🧠 Het AI-Brein Bouwen: Ons Convolutional Neural Network!

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    """
    Onze AI-Dokter! Dit is een Convolutional Neural Network (CNN)
    dat kan leren om longontsteking te herkennen in röntgenfoto's!
    """
    
    def __init__(self):
        super(Net, self).__init__()
        print("🏗️  Bouwen van AI-brein...")
        
        # Laag 1: 32 verschillende "vergrootglazen" (3x3 pixels elk)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        print("   ✅ Laag 1: 32 patroon-detectoren gemaakt")
        
        # Laag 2: 64 nog slimmere "vergrootglazen" 
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        print("   ✅ Laag 2: 64 geavanceerde patroon-detectoren gemaakt")
        
        # Het "brein" deel - hier wordt de finale beslissing gemaakt
        self.fc1 = nn.Linear(in_features=9216, out_features=128)  # Eerste denk-laag
        self.fc2 = nn.Linear(in_features=128, out_features=1)     # Finale beslissing: ziek of gezond?
        print("   ✅ Beslissings-brein gemaakt")

    def forward(self, x):
        """Dit is hoe onze AI 'denkt' wanneer het een foto ziet"""
        # Stap 1: Eerste convolutie + activatie
        x = self.conv1(x)
        x = F.relu(x)  # ReLU = "Rectified Linear Unit" - houdt alleen positieve waarden
        
        # Stap 2: Tweede convolutie + activatie  
        x = self.conv2(x)
        x = F.relu(x)
        
        # Stap 3: Maak het kleiner (pooling)
        x = F.max_pool2d(x, 2)
        
        # Stap 4: Maak het plat voor het breindeel
        x = torch.flatten(x, 1)
        
        # Stap 5: Eerste denklaag
        x = self.fc1(x)
        x = F.relu(x)
        
        # Stap 6: Finale beslissing!
        output = self.fc2(x)
        return output

# Maak onze AI
print("\n🤖 Creëren van onze AI-Dokter...")
net = Net()
print("🎉 AI-Dokter succesvol gebouwd!")

print(f"\n🧮 Fun fact: Onze AI heeft {sum(p.numel() for p in net.parameters()):,} parameters om te leren!")
print("   Dat zijn heel veel getallen die perfect afgesteld moeten worden! 🤯")

## 🎯 De leerregels Instellen

Nu moeten we onze AI leren **hoe** het kan leren! Hiervoor hebben we een paar belangrijke ingrediënten nodig:

### 📏 Loss Functie (Foutmeter):
Dit is hoe we de AI vertellen of het goed of fout zit. We gebruiken **Binary Cross-Entropy Loss**: een fancy naam voor "meet hoe ver je antwoord van de waarheid afzit".

### 🔧 Optimizer (Leeralgoritme): 
Dit is de "leraar" die de AI helpt om beter te worden door te vertellen hoe de kernels aangepast kunnen worden. **Adam** is een slimme leraar die weet wanneer grote en kleine aanpassingen nodig zijn.

Denk aan het als het leren fietsen: de loss functie zegt "je valt om!", en de optimizer zegt "probeer je stuur iets meer naar links te draaien". 🚴‍♀️

In [ ]:
# ⚙️ AI Leerinstellingen Configureren

print("⚙️ Instellen van de AI leer-instellingen...")

# Ons AI model
model = Net()

# Check of we een snelle GPU hebben om op te trainen
if torch.cuda.is_available():
    model.cuda()  # Zet de AI op de GPU voor sneller leren!
    print("🚀 GPU gevonden! Training zal veel sneller gaan!")
    device = 'cuda'
else:
    print("💻 Geen GPU gevonden. Training op CPU (dit kan wat langer duren)")
    device = 'cpu'

# De "leraar" die onze AI helpt verbeteren
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
print("👨‍🏫 Adam optimizer ingesteld (learning rate: 0.0003)")

# De "foutmeter" die meet hoe goed onze AI het doet
loss_function = torch.nn.BCEWithLogitsLoss()
print("📏 Binary Cross-Entropy loss functie ingesteld")

print("\n✅ Alles klaar voor training!")
print("🎓 Onze AI is nu klaar om te leren hoe longontsteking te herkennen!")

In [ ]:
# 🎓 De training 

from tqdm import tqdm

def train_medmnist(model, train_dataloader, val_dataloader, optimizer, epochs, device='cuda', val_freq=1):
    """
    Deze functie is het kloppende hart van onze AI training!
    Het laat onze AI duizenden foto's bekijken en leren van elke fout.
    """
    
    print(f"🎓 Start training voor {epochs} epochs (leer-rondes)!")
    print("📈 Elke epoch laat de AI alle training foto's zien...")
    
    train_loss = []  # Houdt bij hoe goed de AI leert
    val_loss = []    # Houdt bij hoe goed de AI generaliseert
    
    for epoch in tqdm(range(epochs), desc="🧠 AI aan het leren"):
        # AI in leer-modus zetten
        model.train()
        steps = 0
        epoch_loss = 0
        
        # Laat de AI alle training foto's zien
        for batch in train_dataloader:
            # Reset de "les-geheugen"
            optimizer.zero_grad()
            
            # Haal foto's en labels
            images = batch['img'].float().to(device)
            labels = batch['label'].float().to(device)
            
            # Laat de AI een voorspelling maken
            output = model(images.unsqueeze(1)) 
            
            # Bereken hoe fout de AI zat
            loss = loss_function(output, labels)
            epoch_loss += loss.item()
            
            # Leer van de fouten (backpropagation - de magie van AI!)
            loss.backward()
            optimizer.step()
            steps += 1
           
        train_loss.append(epoch_loss/steps)

        # Test hoe goed de AI het doet op ongeziene foto's
        if epoch % val_freq == 0:
            steps = 0
            val_epoch_loss = 0
            model.eval()  # AI in test-modus
            
            with torch.no_grad():  # Geen leren, alleen testen!
                for batch in val_dataloader:
                    images = batch['img'].float().to(device)
                    labels = batch['label'].float().to(device)
                    output = model(images.unsqueeze(1)) 
                    loss = loss_function(output, labels)
                    val_epoch_loss += loss.item()
                    steps += 1
            val_loss.append(val_epoch_loss/steps)

    # Toon hoe de AI heeft geleerd
    plt.figure(figsize=(10, 6))
    plt.plot(train_loss, label='📚 Training Loss (hoe goed leert de AI)', color='blue')
    plt.plot(np.arange(0, epochs, val_freq), val_loss, label='🧪 Validation Loss (hoe goed generaliseert de AI)', color='red')
    plt.xlabel('Epochs (Leer-rondes)')
    plt.ylabel('Loss (Hoe veel fouten)')
    plt.title('🎓 AI Leer-Voortgang')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    print("🎉 Training voltooid!")
    return model, train_loss, val_loss

print("🛠️ Training functie geladen en klaar voor gebruik!")

## 🚀 AI Training Starten

**Dit is het** Na alle voorbereiding gaan we nu onze AI-dokter trainen! 

### 📊 Wat ga je zien?
Straks verschijnt er een grafiek die laat zien hoe onze AI leert:
- **Blauwe lijn (Training Loss):** Hoe goed de AI wordt in het herkennen van foto's die het al gezien heeft
- **Rode lijn (Validation Loss):** Hoe goed de AI is in het herkennen van compleet nieuwe foto's

### 🤔 Vragen om over na te denken:
1. **Overfitting Check:** Als de blauwe lijn naar beneden gaat maar de rode lijn omhoog, dan "leert" de AI de antwoorden uit het hoofd in plaats van echt te begrijpen (dat heet overfitting)

2. **Optimale Training:** Kun je zien op welk punt de AI klaar was met leren? Had het korter gekund dan 100 rondes?

**Let op:** Dit kan 5-10 minuten duren! Perfect moment voor een snack 🍪

In [ ]:
# 🚀  Start de AI Training

print("🎬 En... Actie! De AI training begint!")
print("⏰ Dit kan 5-10 minuten duren - perfect voor een drankje! ☕")
print("📊 Kijk hoe de loss (fouten) steeds kleiner worden...")

# Training instellingen
val_freq = 10  # Check validatie elke 10 epochs
n_epochs = 100  # Train voor 100 leer-rondes

print(f"\n🎯 Training Plan:")
print(f"   📚 {n_epochs} epochs (leer-rondes)")
print(f"   🔍 Validatie check elke {val_freq} epochs")
print(f"   📊 We gebruiken {len(train_dataset):,} training foto's")

# START DE TRAINING! 🚀
model, train_loss, val_loss = train_medmnist(
    model, 
    train_dataloader, 
    val_dataloader, 
    optimizer, 
    epochs=n_epochs, 
    val_freq=val_freq,
    device=device
)

print("\n🎉 WOW! Onze AI-dokter is klaar met leren!")
print("🏥 Het kan nu longontsteking herkennen in röntgenfoto's!")

# Analyse van de resultaten
final_train_loss = train_loss[-1]
final_val_loss = val_loss[-1]

print(f"\n📊 FINALE SCORES:")
print(f"   📚 Training Loss: {final_train_loss:.4f}")
print(f"   🧪 Validation Loss: {final_val_loss:.4f}")

if abs(final_train_loss - final_val_loss) < 0.1:
    print("✅ Geweldig! Geen overfitting - de AI generaliseert goed!")
elif final_val_loss > final_train_loss * 1.5:
    print("⚠️  Er is wat overfitting - de AI leert misschien antwoorden uit het hoofd")
else:
    print("👍 Goede balans tussen leren en generaliseren!")

## 🔍 Wat heeft onze AI geleerd?

Weet je nog die kernels waarmee we eerder experimenteerden? Onze AI heeft er zelf **64 verschillende** uitgevonden om longontsteking te herkennen.
Hierboven maakten wij handmatig kernels om lijnen te detecteren, maar onze AI heeft tijdens training automatisch zijn eigen kernels ontwikkeld. Deze kernels zijn geoptimaliseerd voor het herkennen van longziektepatronen.

### 🎨 Wat ga je zien:
Hieronder zie je 64 kleine plaatjes: dat zijn de kernels of "vergrootglazen" die onze AI heeft geleerd om longontsteking te spotten. Sommige lijken op onze handmatige kernels, andere zijn totaal uniek.

**Fun Challenge:** Kun je raden welke kernels lijken op die we eerder maakten? 🔍

In [ ]:
# 🔍 AI-Kernels ontdekken: Wat heeft onze AI geleerd?

print("🕵️ Tijd om de geheimen van onze AI te ontdekken!")
print("   We gaan kijken naar de kernels die de AI zelf heeft uitgevonden...")

# Kies welke input channel je wilt bekijken (0-31)
input_index = 16
print(f"🎯 Bekijken van kernels die reageren op input channel {input_index}")

# Maak een mooi raster van alle 64 kernels
fig, axs = plt.subplots(8, 8, figsize=(12, 12))
fig.suptitle(f'🧠 AI-Geleerde Kernels van Laag 2 (Input Channel {input_index})', fontsize=16)

for i in range(64):
    # Haal een kernel op uit ons getrainde model
    kernel = model.conv2.weight[i, input_index, :, :].detach().cpu().numpy()
    
    # Bereken de positie in het raster
    row, col = divmod(i, 8)
    
    # Toon de kernel
    axs[row, col].imshow(kernel, clim=[-0.1, 0.1], cmap='RdBu_r')
    axs[row, col].set_title(f'K{i}', fontsize=8)
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()

print(f"\n🤔 Interessante observaties:")
print(f"   • Elke kernel is 3x3 pixels groot")
print(f"   • Rode gebieden = positieve waarden (zoek naar lichte plekken)")
print(f"   • Blauwe gebieden = negatieve waarden (zoek naar donkere plekken)")
print(f"   • Grijze gebieden = ~0 (negeer deze pixels)")

print(f"\n🎮 Probeer verschillende input_index waarden (0-31) om meer kernels te zien")
print(f"💡 Tip: Herken je patronen die lijken op onze handgemaakte kernels?")

## 🎨 Laag 1 Kernels: De eerste detectoren

Dit zijn de allereerste "vergrootglazen" van onze AI - de 32 basis-patroon detectoren. Deze zoeken naar simpele dingen zoals lijnen, hoeken en randen in de röntgenfoto's.

**Interessant feit:** Deze eerste laag kernels lijken vaak het meest op patronen die mensen ook zouden herkennen! 👀

In [ ]:
# 🎨 Laag 1 Kernels: De Basis Patroon Detectoren

print("🔍 Nu bekijken we de allereerste kernels van onze AI...")
print("   Deze zoeken naar basis-patronen zoals lijnen en randen!")

# Check de vorm van de eerste laag kernels
kernel_shape = model.conv1.weight.shape
print(f"\n📐 Kernel informatie:")
print(f"   • Aantal kernels: {kernel_shape[0]}")
print(f"   • Input channels: {kernel_shape[1]}")  
print(f"   • Kernel grootte: {kernel_shape[2]}x{kernel_shape[3]}")

# Maak een 4x8 raster voor alle 32 kernels
fig, axs = plt.subplots(4, 8, figsize=(16, 8))
fig.suptitle('🎯 AI-Geleerde Basis Kernels (Laag 1)', fontsize=16)

for i in range(32):
    # Haal kernel i op uit de eerste laag
    kernel = model.conv1.weight[i, 0, :, :].detach().cpu().numpy()
    
    # Bereken positie in het raster
    row, col = divmod(i, 8)
    
    # Visualiseer de kernel
    im = axs[row, col].imshow(kernel, clim=[-0.1, 0.1], cmap='RdBu_r')
    axs[row, col].set_title(f'Kernel {i}', fontsize=10)
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()

print("\n🤓 Deze basis-kernels zijn de bouwstenen!")
print("   🔴 Rode delen zoeken naar lichte pixels")
print("   🔵 Blauwe delen zoeken naar donkere pixels")  
print("   ⚪ Grijze delen worden genegeerd")

print("\n💡 Herken je bekende patronen?")
print("   • Verticale/horizontale lijn detectoren")
print("   • Hoek detectoren")  
print("   • Rand detectoren")
print("   • Blob detectoren")

## 🏥 Hoe goed is onze AI-dokter? Performance evaluatie!

**Het moment van waarheid** 🎭 Onze AI heeft geleerd, maar hoe goed is het eigenlijk? In de echte wereld hangt er veel van af: we willen geen zieke mensen missen, maar ook geen gezonde mensen onnodig bang maken.

### 📊 De Belangrijkste Meetwaarden:

**🎯 Precision (Precisie):**
- Van alle mensen die de AI als "ziek" bestempelt, hoeveel zijn er echt ziek?
- Hoge precisie = weinig "valse alarmen"

**🔍 Recall (Herinnering):** 
- Van alle zieke mensen, hoeveel heeft de AI gevonden?
- Hoge recall = weinig gemiste ziektes

### 🤔 De Balans:
- **Perfect precision maar lage recall:** AI is heel voorzichtig maar mist zieke mensen
- **Perfect recall maar lage precision:** AI vindt alle zieken maar geeft vaak vals alarm dat een gezond persoon ziek is


![Precision vs Recall](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/525px-Precisionrecall.svg.png)

In [ ]:
# 🔮 AI-Dokter in Actie: Voorspellingen bekijken

def validation_results_visualize(model, dataset):
    """
    Laat zien hoe onze AI-dokter een diagnose stelt!
    Vergelijk de echte diagnose met wat onze AI denkt.
    """
    # Kies een willekeurige foto
    index = np.random.randint(0, len(dataset))
    image = dataset[index]['img']
    
    # Maak de foto mooi zichtbaar
    plt.figure(figsize=(8, 6))
    plt.imshow(image.numpy().squeeze(), cmap='gray')
    
    # Haal de echte diagnose op
    image = image.float().to(device)
    label = dataset[index]['label'].item()
    
    # Laat onze AI een voorspelling maken!
    with torch.no_grad():
        model.eval()
        output = F.sigmoid(model(image.view(1, 1, 28, 28))).squeeze()
        ai_prediction = int(output > 0.5)  # > 0.5 = ziek, < 0.5 = gezond
        confidence = output.item()
    
    # Maak een mooie titel met de resultaten
    real_diagnosis = "🔴 Longontsteking" if label == 1 else "💚 Gezond"
    ai_diagnosis = "🔴 Longontsteking" if ai_prediction == 1 else "💚 Gezond"
    
    # Bepaal of de AI het goed had
    if label == ai_prediction:
        result_icon = "✅ CORRECT!"
        title_color = 'green'
    else:
        result_icon = "❌ FOUT!"
        title_color = 'red'
    
    plt.title(f'{result_icon}\nEchte diagnose: {real_diagnosis}\nAI diagnose: {ai_diagnosis}\nZekerheid: {confidence:.2%}', 
              fontsize=12, color=title_color)
    plt.yticks([]) 
    plt.xticks([]) 
    plt.show()
    
    return label == ai_prediction

print("🔮 AI-Dokter Voorspelling Functie geladen!")

In [ ]:
# 🎲 AI-Dokter Performance Test: 10 Willekeurige Diagnoses!

print("🏥 Laten we onze AI-dokter 10 willekeurige patiënten laten bekijken!")
print("   Kijk goed naar elke diagnose - klopt het wat de AI zegt?")

correct_predictions = 0
total_predictions = 10

print("\n" + "="*60)
print("🔍 AI-DOKTER DIAGNOSE SESSIE")
print("="*60)

for i in range(total_predictions):
    print(f"\n👤 Patiënt #{i+1}:")
    is_correct = validation_results_visualize(model, val_dataset)
    
    if is_correct:
        correct_predictions += 1
        print("   ✅ AI-diagnose is correct!")
    else:
        print("   ❌ AI-diagnose is incorrect!")

# Bereken de accuracy
accuracy = (correct_predictions / total_predictions) * 100

print("\n" + "="*60)
print("📊 SESSION RESULTATEN:")
print("="*60)
print(f"🎯 Correcte diagnoses: {correct_predictions} van {total_predictions}")
print(f"📈 Session Accuracy: {accuracy:.1f}%")

if accuracy >= 80:
    print("🌟 Geweldig! Onze AI-dokter presteert uitstekend!")
elif accuracy >= 70:
    print("👍 Goed! Onze AI-dokter doet het aardig goed!")
elif accuracy >= 60:
    print("📚 Oké - onze AI kan nog wel wat meer leren...")
else:
    print("🎓 Hmm, onze AI heeft nog meer training nodig!")

print(f"\n💡 Tip: Run deze cel opnieuw voor andere willekeurige patiënten!")

In [ ]:
# 📊 Precision & Recall Calculator: Hoe Goed is Onze AI Echt?

def get_precision_recall(model, dataloader):
    """
    Berekent hoe precies en volledig onze AI-dokter is.
    
    Returns:
    - Precision: Van alle "ziek" diagnoses, hoeveel zijn correct?
    - Recall: Van alle zieke mensen, hoeveel heeft de AI gevonden?
    """
    print("🔬 Bezig met nauwkeurige evaluatie van onze AI-dokter...")
    
    model.eval()
    
    # Counters voor alle mogelijke uitkomsten
    TP = 0  # True Positives: AI zegt ziek, is ook ziek ✅
    TN = 0  # True Negatives: AI zegt gezond, is ook gezond ✅  
    FP = 0  # False Positives: AI zegt ziek, maar is gezond ❌ (vals alarm)
    FN = 0  # False Negatives: AI zegt gezond, maar is ziek ❌ (gemist!)
    
    total_patients = 0
    
    with torch.no_grad():
        for data in dataloader:
            images = data['img'].float().to(device)
            labels = data['label'].squeeze()
            total_patients += len(labels)
            
            # AI voorspellingen maken
            output = F.sigmoid(model(images.unsqueeze(1))).squeeze().cpu()
            pred_classes = (output >= 0.5).to(torch.int8)  # 0.5 is de drempel
            
            # Tel alle uitkomsten
            TP += (pred_classes * labels).sum().item()
            TN += ((1 - pred_classes) * (1 - labels)).sum().item()
            FP += (pred_classes * (1 - labels)).sum().item()
            FN += ((1 - pred_classes) * labels).sum().item()
    
    # Bereken de metrics
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    accuracy = (TP + TN) / total_patients
    
    print(f"📊 GEDETAILLEERDE RESULTATEN:")
    print(f"   👥 Totaal aantal patiënten getest: {total_patients}")
    print(f"   ✅ True Positives (correct ziek): {TP}")
    print(f"   ✅ True Negatives (correct gezond): {TN}")
    print(f"   ❌ False Positives (vals alarm): {FP}")
    print(f"   ❌ False Negatives (gemist ziek): {FN}")
    
    return precision, recall, accuracy, TP, TN, FP, FN

print("📊 Precision & Recall calculator geladen!")

In [ ]:
# 🏆 FINALE EVALUATIE: Hoe Goed is Onze AI-Dokter?

print("🏥 Tijd voor de ultieme test!")
print("   We gaan onze AI-dokter evalueren op een compleet nieuwe test-dataset...")

# Laad de test dataset - foto's die de AI nog NOOIT heeft gezien!
test_dataset = MedMNISTData(medmnist.PneumoniaMNIST(split='test', download=False))
test_loader = monai.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"🧪 Test dataset geladen: {len(test_dataset)} compleet nieuwe foto's!")

# Voer de volledige evaluatie uit
precision, recall, accuracy, TP, TN, FP, FN = get_precision_recall(model, test_loader)

print(f"\n🏆 FINALE RESULTATEN VAN ONZE AI-DOKTER:")
print("="*50)
print(f"🎯 Precision (Nauwkeurigheid): {precision:.3f} ({precision*100:.1f}%)")
print(f"🔍 Recall (Volledigheid): {recall:.3f} ({recall*100:.1f}%)")  
print(f"📊 Overall Accuracy: {accuracy:.3f} ({accuracy*100:.1f}%)")

print(f"\n🤔 Wat betekent dit?")
print(f"   🎯 Van elke 100 'ziek' diagnoses zijn er {precision*100:.0f} correct")
print(f"   🔍 Van elke 100 zieke patiënten vindt onze AI er {recall*100:.0f}")
print(f"   📊 Van alle diagnoses is {accuracy*100:.0f}% correct")

# Interpretatie geven
print(f"\n💭 INTERPRETATIE:")
if precision > 0.8 and recall > 0.8:
    print("🌟 UITSTEKEND! Onze AI zou echt kunnen helpen in ziekenhuizen!")
elif precision > 0.7 and recall > 0.7:
    print("👍 GOED! Met wat meer training zou dit een nuttige tool kunnen zijn!")
elif precision > 0.6 or recall > 0.6:
    print("📚 OKAY! De AI heeft potentie maar heeft meer data en training nodig.")
else:
    print("🎓 BEGINNERSNIVEAU! De AI heeft veel meer leermateriaal nodig.")

# Specifieke medische context
if recall > precision:
    print(f"🔍 Onze AI is beter in het VINDEN van ziekte dan in nauwkeurige diagnoses")
    print(f"   → Goed voor screening (eerste check)")
else:
    print(f"🎯 Onze AI is nauwkeuriger dan volledig")  
    print(f"   → Goed voor confirmatie (tweede mening)")

## 🤔 Reflectie: Wat vind je van de resultaten?

**Denk er eens over na:** Onze AI heeft zojuist duizenden röntgenfoto's beoordeeld. Wat vind je van de precision en recall scores?

### 🧠 Belangrijke Vragen:
1. **Is 80% accuracy goed genoeg voor medische diagnoses?**
2. **Wat zou er gebeuren als onze AI 20% van de longontstekingen mist?**
3. **Zou je willen dat een AI jouw röntgenfoto beoordeelt?**

### 💭 Context:
- Menselijke radiologen hebben ongeveer 85-95% accuracy bij longontsteking diagnoses
- Maar zij worden ook moe, onze AI niet!
- In de echte wereld wordt AI vaak gebruikt als "tweede mening" naast een dokter
- Echte medische AI is gebaseerd op veel grotere en gedetailleerdere foto's dan die wij gebruikten, en kan daardoor beter longontstekingen vinden

**Wat denk jij? Vertrouw je onze AI-dokter?** 🤖👨‍⚕️

## 🔍 Confusion Matrix

**Wat is een Confusion Matrix?** 🤯 Het is een handige tabel die PRECIES laat zien waar onze AI goed in is en waar het fouten maakt.

### 📊 Hoe lees je het:
- **Linksboven:** Gezonde mensen die terecht als gezond gediagnosticeerd zijn ✅
- **Rechtsonder:** Zieke mensen die terecht als ziek gediagnosticeerd zijn ✅  
- **Rechtsboven:** Gezonde mensen die ten onrechte als ziek bestempeld zijn ❌ (Vals alarm!)
- **Linksonder:** Zieke mensen die gemist zijn ❌ (Gevaarlijk!)

### 🏥 Medische Impact:
- **Vals alarm (rechtsboven):** Onnodige stress en extra tests
- **Gemiste diagnose (linksonder):** Potentieel levensgevaarlijk!

**Challenge:** Kun je de confusion matrix ook voor de validatie data maken? Die loader heet `val_dataloader`! 🎯

In [ ]:
# 🔍 Confusion Matrix: De Waarheids-Tabel van Onze AI!

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, dataloader, dataset_name="Test"):
    """
    Maakt een prachtige confusion matrix die laat zien waar onze AI goed en slecht in is!
    
    Parameters:
    - model: onze getrainde AI-dokter
    - dataloader: de data om te testen  
    - dataset_name: naam voor de titel
    """
    print(f"🔬 Maken van confusion matrix voor {dataset_name} dataset...")
    
    model.eval()
    true_labels = []
    predicted_labels = []
    confidences = []

    with torch.no_grad():
        for data in dataloader:
            images = data['img'].float().to(device)
            labels = data['label'].squeeze()
            
            # AI voorspellingen
            output = F.sigmoid(model(images.unsqueeze(1))).squeeze().cpu()
            pred_classes = (output >= 0.5).to(torch.int8)
            
            # Bewaar alle resultaten
            true_labels.extend(labels.numpy())
            predicted_labels.extend(pred_classes.numpy())
            confidences.extend(output.numpy())

    # Maak de confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels, labels=[0, 1])
    
    # Maak een mooie visualisatie
    plt.figure(figsize=(10, 8))
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, 
        display_labels=['💚 Gezond', '🔴 Longontsteking']
    )
    disp.plot(cmap='Blues', values_format='d')
    
    plt.title(f'🔍 AI-Dokter Confusion Matrix ({dataset_name} Data)', fontsize=16)
    plt.xlabel('AI Voorspelling', fontsize=12)
    plt.ylabel('Werkelijke Diagnose', fontsize=12)
    
    # Voeg interpretatie toe
    tn, fp, fn, tp = cm.ravel()
    plt.figtext(0.02, 0.02, 
                f"✅ Correct Gezond: {tn} | ✅ Correct Ziek: {tp}\n"
                f"❌ Vals Alarm: {fp} | ❌ Gemist Ziek: {fn}", 
                fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
    
    plt.tight_layout()
    plt.show()
    
    # Geef interpretatie
    total = tn + fp + fn + tp
    print(f"\n📊 CONFUSION MATRIX ANALYSE:")
    print(f"   ✅ Totaal correct: {tn + tp} van {total} ({((tn + tp)/total)*100:.1f}%)")
    print(f"   💚 Correct gezond: {tn}")
    print(f"   🔴 Correct ziek: {tp}")
    print(f"   ⚠️  Vals alarm: {fp} (gezond → ziek)")
    print(f"   🚨 Gemist ziek: {fn} (ziek → gezond)")
    
    if fn > fp:
        print(f"\n⚠️  Onze AI mist meer ziektes dan het valse alarmen geeft!")
        print(f"    → Dit kan gevaarlijk zijn in echte medische situaties")
    elif fp > fn:
        print(f"\n📢 Onze AI geeft meer valse alarmen dan dat het ziektes mist")
        print(f"    → Veiliger, maar kan onnodige stress veroorzaken")
    else:
        print(f"\n⚖️  Goede balans tussen vals alarm en gemiste diagnoses!")

# VOER DE ANALYSE UIT!
print("🎯 Confusion Matrix voor Test Data:")
plot_confusion_matrix(model, test_loader, "Test")

print(f"\n💡 TIP: Probeer ook 'plot_confusion_matrix(model, val_dataloader, \"Validation\")' voor de validatie data!")